# Bioinformatics for Genome Medicine Workshop

###*Prof. Jason Wong, School of Biomedical Sciences, HKU*

####The objective of this lecture is to demonstrate how next-generation sequencing data is aligned to the reference genome sequence and how variant calling can be performed.


## *** Package installation and downloads for workshop (~ 5 minutes)

1.   conda (for simple installation of packages)
2.   FastQC (for reads quality check)
3.   bwa (tools for sequence alignment)
4.   samtools (tools for processing sam & bam files)  

**IMPORTANT：Every time you connect to Google Colab, you have to perform these set up steps again.**

In [ ]:
# set working pathway to your own google drive doc (~ 1 min)
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
# install conda (~ 1 min). There will be a message saying that the session has crashed, but don't worry about this. This is due to the session restarting following conda installation.
!pip install -q condacolab
import condacolab#
condacolab.install()

In [ ]:
# install fastqc (~ 2 mins)
!conda install -c bioconda fastqc

In [ ]:
# install bwa (~ 1 mins)
!conda install -c bioconda bwa

In [ ]:
#Install igv-notebook
!pip install igv-notebook

In [ ]:
# install lofreq (~ 1 mins)
!conda install -c bioconda lofreq

In [ ]:
# install samtools (~1 mins)
!conda install -c bioconda samtools

In [ ]:
# install bedtools (~ 1 mins)
!apt install bedtools

## Set working directory

By default, the working directory will be My Drive/PB_course

In [ ]:
import os
try:
  os.mkdir("/content/gdrive/My Drive/PB_course")         # change this path if necessary
except FileExistsError:
  print("directory already exist. OK to continue")
os.chdir("/content/gdrive/My Drive/PB_course")

## Download ready prepared files for analysis.

In [ ]:
#download reference sequence
# double check that we are in the right directory (~ 30s)
import os
os.chdir("/content/gdrive/MyDrive/PB_course")                     # change this path if necessary

import os
if os.path.isfile("/content/gdrive/MyDrive/PB_course/DB_trunc/chr2.fa"):    # check if the file exist
  print("reference file already exit, OK to continue.")
else:
  !wget -O DB_trunc.zip "https://drive.usercontent.google.com/download?id=1zF8BTkR90kEr23b3NecES3u8akbSfIw9&export=download&authuser=0&confirm=t&uuid=ec27dd21-2518-4a97-8547-7cada1fde88e&at=APZUnTXxF4K7otx7GhcCftn0sRfA%3A1709043759277"  # download if file not exist
  #unzip fasta file
  !unzip DB_trunc.zip
  !rm DB_trunc.zip

!ls -l ./DB_trunc/

In [ ]:
#download sample sequences
import os
os.chdir("/content/gdrive/My Drive/PB_course/")
if os.path.isfile("/content/gdrive/MyDrive/PB_course/Datasets/ChIP-seq_H3K27ac_example.fq.gz"):    # check if the file exist
  print("file already exit, OK to continue.")
else:
 !wget -O Datasets.zip https://github.com/jasonwong-lab/HKU-Practical-Bioinformatics/raw/main/files/Datasets.zip    # download necessary file
 !unzip -o Datasets.zip   #unzip file
 !rm Datasets.zip

## NGS_alignment command line

1.1 Quality control

1.2. Burrows–Wheeler Aligner

1.3. Working with SAM files

1.4. Align the WGS paired-end file

In [ ]:
#Check what datasets we have downloaded
%cd /content/gdrive/MyDrive/PB_course
!ls -l

%cd Datasets/
!ls -l

In [ ]:
# To look at the ChIP-seq fastq file type (head -n 12 to print first 12 lines):
!zcat < ChIP-seq_H3K27ac_example.fq.gz |head -n 12

In [ ]:
# How many reads are there in the file? Type your code below:
!zcat < ChIP-seq_H3K27ac_example.fq.gz | wc -l

### Quality control

In [ ]:
## Run FastQC
!fastqc ChIP-seq_H3K27ac_example.fq.gz

In [ ]:
# check resulting html file
!ls

#download file and check html file with your local browser
from google.colab import files
files.download('ChIP-seq_H3K27ac_example_fastqc.html')

you can also download the file **ChIP-seq_H3K27ac_example_fastqc.html** from "gdrive/MyDrive/PB_course/Datasets" on the left side

### Burrows–Wheeler Aligner

In [ ]:
# Look at the options for bwa and bwa mem
!bwa

In [ ]:
!bwa mem

In [ ]:
# let's take a look at the database files
%cd /content/gdrive/My Drive/PB_course/DB_trunc

!ls -l

In [ ]:
#Get ready to run BWA. First Go into Datasets directory
%cd /content/gdrive/My Drive/PB_course/Datasets

In [ ]:
# make an directory to store output file
!mkdir BAM

In [ ]:
#Do sequence alignment
!bwa mem ../DB_trunc/chr2.fa ./ChIP-seq_H3K27ac_example.fq.gz > ./BAM/ChIP-seq_H3K27ac_example.sam

In [ ]:
# check the result
!head -n 20 ./BAM/ChIP-seq_H3K27ac_example.sam

### Working with SAM files

In [ ]:
#Check the samtools command
!samtools

In [ ]:
#Check out some stats about our aligned file
!samtools flagstat ./BAM/ChIP-seq_H3K27ac_example.sam

In [ ]:
#Prepare the file for viewing on genome browser:
#Step 1 – Convert SAM to BAM
!samtools view -b ./BAM/ChIP-seq_H3K27ac_example.sam > ./BAM/ChIP-seq_H3K27ac_example.bam

In [ ]:
#Step 2 – sort BAM file
!samtools sort ./BAM/ChIP-seq_H3K27ac_example.bam > ./BAM/ChIP-seq_H3K27ac_example_sorted.bam

In [ ]:
#Step 3 – index BAM file
!samtools index ./BAM/ChIP-seq_H3K27ac_example_sorted.bam

In [ ]:
#Look at the files that we have created
!ls -l ./BAM/

### IGV browser

In [ ]:
#Load track from local paths
import igv_notebook

igv_notebook.init()

b = igv_notebook.Browser(
    {
        "genome": "hg38",
        "locus": "chr2:47,782,081-47,807,953"
    }
)

b.load_track(
    {
        "name": "ChIP-seq_H3K27ac",
        "path": "./BAM/ChIP-seq_H3K27ac_example_sorted.bam",
        "indexPath": "./BAM/ChIP-seq_H3K27ac_example_sorted.bam.bai",
        "format": "bam",
        "type": "alignment"
    })

### Align the WGS paired-end file

In [ ]:
# Check that we have the WGS fastq files
%cd /content/gdrive/My Drive/PB_course/Datasets
!ls -l

In [ ]:
# If you don't have WGS_example_1.fq.gz or WGS_example_2.fq.gz, run this cell to redownload the files
import os
os.chdir("/content/gdrive/My Drive/PB_course/")

import os
if os.path.isfile("/content/gdrive/MyDrive/PB_course/Datasets/WGS_example_1.fq.gz"):    # check if the file exist
  print("reference file already exit, OK to continue.")
else:
 !wget -O Datasets.zip https://github.com/jasonwong-lab/HKU-Practical-Bioinformatics/raw/main/files/Datasets.zip
 !unzip -o Datasets.zip   #unzip file
 !rm Datasets.zip

#current directory should still be ~/Datasets
%cd Datasets/
!ls -l

In [ ]:
#look at our work directory and paired-end file:
!zcat < WGS_example_1.fq.gz |head -1
!zcat < WGS_example_2.fq.gz |head -1

In [ ]:
#align the WGS paired-end file:( ~ 1 mins)
#Step 1 – Aligning paired-end file using bwa mem:
!bwa mem ../DB_trunc/chr2.fa ./WGS_example_1.fq.gz ./WGS_example_2.fq.gz > ./BAM/WGS_example.sam


In [ ]:
#Step 2 – Output sorted BAM, this time use piping to skip one step:
!samtools view -b ./BAM/WGS_example.sam | samtools sort >./BAM/WGS_example_sorted.bam

In [ ]:
#Step 3 – Index sorted bam file
!samtools index ./BAM/WGS_example_sorted.bam

In [ ]:
#Check stats about the aligned WGS file
!samtools flagstat ./BAM/WGS_example_sorted.bam

In [ ]:
#IGV
#Load track from local paths
import igv_notebook

igv_notebook.init()

b = igv_notebook.Browser(
    {
        "genome": "hg38",
        "locus": "chr2:47,782,081-47,807,953"
    }
)

b.load_track(
    {
        "name": "ChIP-seq H3K27ac",
        "path": "./BAM/ChIP-seq_H3K27ac_example_sorted.bam",
        "indexPath": "./BAM/ChIP-seq_H3K27ac_example_sorted.bam.bai",
        "format": "bam",
        "type": "alignment"
    })

b.load_track(
    {
        "name": "WGS",
        "path": "./BAM/WGS_example_sorted.bam",
        "indexPath": "./BAM/WGS_example_sorted.bam.bai",
        "format": "bam",
        "type": "alignment"
    })

# Aligned and do variant calling on WXS data

In [ ]:
# Run this cell to download the WXS files
import os
os.chdir("/content/gdrive/My Drive/PB_course/Datasets")

import os
if os.path.isfile("/content/gdrive/MyDrive/PB_course/Datasets/WXS_example_cancer_1.fq.gz"):    # check if the file exist
  print("WXS file already exit, OK to continue.")
else:
 !wget -O Datasets_WXS.zip "https://drive.usercontent.google.com/download?id=1pivDkbhXEYN57f-1pduYR5E2TtwvZGsx&export=download&authuser=0&confirm=t&uuid=6ccaeb5f-3091-48da-ac04-3afc5256c499&at=APZUnTXUz8J8VXoZgzVPVXDUlJHz%3A1709081319052"
 !unzip -o Datasets_WXS.zip   #unzip file
 !rm Datasets_WXS.zip


#Check what files we have now
%cd /content/gdrive/MyDrive/PB_course/Datasets/
!ls -l

In [ ]:
#align the WXS paired-end file:( ~ 3 mins)
#Step 1 – Aligning normal and cancer paired-end file using bwa mem:
!bwa mem ../DB_trunc/chr2.fa ./WXS_example_normal_1.fq.gz ./WXS_example_normal_2.fq.gz > ./BAM/WXS_example_normal.sam
!bwa mem ../DB_trunc/chr2.fa ./WXS_example_cancer_1.fq.gz ./WXS_example_cancer_2.fq.gz > ./BAM/WXS_example_cancer.sam

In [ ]:
#Step 2 – Output sorted BAM, this time use piping to skip one step:
!samtools view -b ./BAM/WXS_example_normal.sam | samtools sort >./BAM/WXS_example_normal_sorted.bam
!samtools view -b ./BAM/WXS_example_cancer.sam | samtools sort >./BAM/WXS_example_cancer_sorted.bam

In [ ]:
#Step 3 – Index sorted bam file
!samtools index ./BAM/WXS_example_normal_sorted.bam
!samtools index ./BAM/WXS_example_cancer_sorted.bam

In [ ]:
#IGV
#Load track from local paths
import igv_notebook

igv_notebook.init()

b = igv_notebook.Browser(
    {
        "genome": "hg38",
        "locus": "chr2:47,782,081-47,807,953"
    }
)

b.load_track(
    {
        "name": "WXS Normal",
        "path": "./BAM/WXS_example_normal_sorted.bam",
        "indexPath": "./BAM/WXS_example_normal_sorted.bam.bai",
        "format": "bam",
        "type": "alignment"
    })

b.load_track(
    {
        "name": "WXS Cancer",
        "path": "./BAM/WXS_example_cancer_sorted.bam",
        "indexPath": "./BAM/WXS_example_cancer_sorted.bam.bai",
        "format": "bam",
        "type": "alignment"
    })

#Variant calling - call normal and tumour separately then subtract

In [ ]:
!lofreq

In [ ]:
# Variant calling using LoFreq
!lofreq call

In [ ]:
#Step 1 – Create directory to store VCF files. Make sure we are in the Datasets directory.
%cd /content/gdrive/My Drive/PB_course/Datasets/
!mkdir VCF/

In [ ]:
#Step 2 – Run LoFreq call
!rm VCF/*.vcf
!lofreq call ./BAM/WXS_example_normal_sorted.bam -o ./VCF/WXS_normal_nofilter.vcf -f ../DB_trunc/chr2.fa --verbose --no-default-filter
!lofreq call ./BAM/WXS_example_cancer_sorted.bam -o ./VCF/WXS_cancer_nofilter.vcf -f ../DB_trunc/chr2.fa --verbose --no-default-filter

In [ ]:
#Step 3 - Run LoFreq filter
!lofreq filter --print-all -i ./VCF/WXS_normal_nofilter.vcf -o ./VCF/WXS_normal_filter.vcf
!lofreq filter --print-all -i ./VCF/WXS_cancer_nofilter.vcf -o ./VCF/WXS_cancer_filter.vcf

In [ ]:
#check result:
!head -n 30 ./VCF/WXS_normal_filter.vcf

In [ ]:
#Rerun lofreq filter again to only print PASS variants
!lofreq filter -i ./VCF/WXS_normal_nofilter.vcf -o ./VCF/WXS_normal_PASS.vcf
!lofreq filter -i ./VCF/WXS_cancer_nofilter.vcf -o ./VCF/WXS_cancer_PASS.vcf

In [ ]:
#Count the number of lines in WXS_normal_PASS.vcf and WXS_cancer_PASS.vcf Remove the header by grep
!grep -v '#' ./VCF/WXS_normal_PASS.vcf | wc -l
!grep -v '#' ./VCF/WXS_cancer_PASS.vcf | wc -l

In [ ]:
#Get a list of mutations only present in the cancer sample
!subtractBed -a ./VCF/WXS_cancer_PASS.vcf -b ./VCF/WXS_normal_PASS.vcf > ./VCF/WXS_cancer_only_PASS.vcf

In [ ]:
#Count the number of lines in WXS_cancer_only_PASS.vcf Remove the header by grep
!grep -v '#' ./VCF/WXS_cancer_only_PASS.vcf | wc -l

### Check the functional impact of variants

VEP online: http://useast.ensembl.org/Tools/VEP



In [ ]:
#download the germline (WXS_normal_PASS.vcf) variants for upload to VEP

from google.colab import files
files.download('./VCF/WXS_normal_PASS.vcf')


In [ ]:
#download the somatic (WXS_cancer_only_PASS.vcf) variants for upload to VEP
files.download('./VCF/WXS_cancer_only_PASS.vcf')